In [1]:
# import sqlalchemy
# sqlalchemy.__version__

In [2]:
# Import packages
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd


In [3]:
# Create engine: engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [4]:
#checking table names
inspector = inspect(engine)
inspector.get_table_names()

['Bookings', 'Facilities', 'Members']

# Start of Questions and Solutions :)

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [5]:
sql_10 = (r"""

SELECT
f.name AS facility_name
, SUM(CASE WHEN b.memid != 0 THEN f.membercost * b.slots
WHEN b.memid = 0 THEN f.guestcost * b.slots END) as total_revenue
FROM Bookings b
JOIN Facilities f
ON b.facid = f.facid
GROUP BY f.name
HAVING SUM(CASE WHEN b.memid != 0 THEN f.membercost * b.slots
WHEN b.memid = 0 THEN f.guestcost * b.slots END) < 1000
ORDER BY 2 DESC


""")

In [6]:
pd.read_sql_query(sql_10, engine)

,facility_name,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [7]:
sql_11 = (r"""

SELECT 
 (m.surname ||", "|| m.firstname) AS name_of_member
, CASE WHEN m1.memid != 0 THEN (m1.surname|| ", "|| m1.firstname) 
ELSE "" END AS recommended_by

FROM Members m
LEFT JOIN Members m1
ON m.recommendedby = m1.memid
WHERE m.memid !=0 
ORDER BY m.surname, m.firstname, m1.surname, m1.firstname


""")

In [8]:
pd.read_sql_query(sql_11, engine)

,name_of_member,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",
9,"Farrell, Jemima",


/* Q12: Find the facilities with their usage by member, but not guests */

In [9]:
sql_12 = (r"""

SELECT
f.name
--, SUM(b.slots) AS member_only_usage_by_number_of_slots 
--initially I had calculated usage by slots but changed to members only booking count based on a slack thread

, COUNT(DISTINCT b.bookid) AS members_only_booking_count
FROM Bookings b
JOIN Facilities f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name
ORDER BY 2 DESC

""")

In [10]:
pd.read_sql_query(sql_12, engine)

,name,members_only_booking_count
0,Pool Table,783
1,Snooker Table,421
2,Massage Room 1,421
3,Table Tennis,385
4,Badminton Court,344
5,Tennis Court 1,308
6,Tennis Court 2,276
7,Squash Court,195
8,Massage Room 2,27


/* Q13: Find the facilities usage by month, but not guests */

In [11]:
sql_13 = (r"""


SELECT
CASE strftime('%m', b.starttime) when '01' then 'January' when '02' then 'Febuary' when '03' then 'March' when '04' then 'April' when '05' then 'May' when '06' then 'June' when '07' then 'July' when '08' then 'August' when '09' then 'September' when '10' then 'October' when '11' then 'November' when '12' then 'December' else '' end
as month 
--, SUM(b.slots) AS member_only_usage_by_number_of_slots
--initially I had calculated usage by slots but changed to members only booking count based on a slack thread

, COUNT(DISTINCT b.bookid) AS members_only_booking_count

FROM Bookings b
JOIN Facilities f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY month
ORDER BY 2 DESC

""")

In [12]:
pd.read_sql_query(sql_13, engine)

,month,members_only_booking_count
0,September,1512
1,August,1168
2,July,480
